In [1]:
QFILE=`mktemp /dev/mqueue/mqtest.XXXX`

In [2]:
echo $QFILE

/dev/mqueue/mqtest.nPyf


In [3]:
ls -l $QFILE

-rw------- 1 jnewsome jnewsome 80 May  9 17:38 /dev/mqueue/mqtest.nPyf


In [4]:
cat $QFILE

QSIZE:0          NOTIFY:0     SIGNO:0     NOTIFY_PID:0     


In [5]:
QNAME="/$(basename $QFILE)"
echo $QNAME

/mqtest.nPyf


In [6]:
read -r -d '' C_PRELIMINARIES << EOM
#include <fcntl.h>
#include <sys/stat.h>
#include <mqueue.h>
#include <stdio.h>
#include <stdlib.h>

#define CHECK_GTE0(x) { \
  if ((x) < 0) {\
    perror(#x);\
    exit(EXIT_FAILURE);\
  }\
}

#define CHECK_EQ0(x) { \
  if ((x) != 0) {\
    perror(#x);\
    exit(EXIT_FAILURE);\
  }\
}
EOM

gcc -o mq_getattr -xc - -lrt <<EOF
$C_PRELIMINARIES

int main(int argc, char **argv) {
  mqd_t q;
  CHECK_GTE0(q = mq_open(argv[1], O_WRONLY));
  struct mq_attr attr;
  CHECK_EQ0(mq_getattr(q, &attr));
  printf("flags: %ld\n", attr.mq_flags);
  printf("maxmsg: %ld\n", attr.mq_maxmsg);
  printf("msgsize: %ld\n", attr.mq_msgsize);
  printf("curmsgs: %ld\n", attr.mq_curmsgs);
}
EOF

In [7]:
./mq_getattr $QNAME

flags: 0
maxmsg: 10
msgsize: 8192
curmsgs: 0


In [8]:
gcc -o mq_send -xc - -lrt <<EOF
$C_PRELIMINARIES

int main(int argc, char **argv) {
  mqd_t q;
  CHECK_GTE0(q = mq_open(argv[1], O_WRONLY));
  struct mq_attr attr;
  CHECK_EQ0(mq_getattr(q, &attr));
  
  char *buf = malloc(attr.mq_msgsize);
  size_t n;
  CHECK_GTE0(n = fread(buf, 1, attr.mq_msgsize, stdin));
  CHECK_GTE0(mq_send(q, buf, n, 0));
}
EOF

In [9]:
./mq_send $QNAME <<EOF
Hello message queue
Secondline
EOF

In [10]:
cat $QFILE

QSIZE:31         NOTIFY:0     SIGNO:0     NOTIFY_PID:0     


In [11]:
./mq_getattr $QNAME

flags: 0
maxmsg: 10
msgsize: 8192
curmsgs: 1


In [12]:
gcc -o mq_recv -xc - -lrt <<EOF
$C_PRELIMINARIES

int main(int argc, char **argv) {
  mqd_t q;
  CHECK_GTE0(q = mq_open(argv[1], O_RDONLY));
  struct mq_attr attr;
  CHECK_EQ0(mq_getattr(q, &attr));
  
  char *buf = malloc(attr.mq_msgsize);
  ssize_t n;
  int priority;
  CHECK_GTE0(n = mq_receive(q, buf, attr.mq_msgsize, &priority));
  CHECK_GTE0(fwrite(buf, 1, n, stdout));
}
EOF

In [13]:
./mq_recv $QNAME

Hello message queue
Secondline
